In [2]:
import pandas as pd
import numpy as np
import json 
import re
from geojson import Feature, FeatureCollection, MultiPoint, Point
import uuid
import math
from operator import attrgetter


In [3]:
class Vehicle:
    def __init__(
        self,
        id,
        department,
        make,
        model,
        year,
        vehicle_class
        ):
        self.id = id
        self.department = department
        self.make = make
        self.model = model
        self.year = year
        self.vehicle_class = vehicle_class
        self.trips = []

In [4]:
class Trip:
    def __init__(
        self,
        vehicle,
        start_time,
        end_time,
        date,
        ):
        self.id = str(uuid.uuid4())
        self.vehicle = vehicle
        self.start_time = start_time
        self.end_time = end_time
        self.date = date
        self.stops = []
        self.num_stops = 0
        self.distance = 0
        self.total_duration = 0
        self.idle_duration = 0
        self.driving_duration = 0
        self.stopped_duration = 0
        self.geojson = []
        self.show = True

    def add_stop(self, stop):
        #self.stops.append(stop.__dict__)
        stop_properties = {
            'vehicle': self.vehicle,
            'stopDuration': stop.stop_duration,
            'idleDuration': stop.idle_duration,
            'startTime': stop.start_time,
            'endTime': stop.end_time,
            'drivingDuration': stop.driving_duration,
            'distance': stop.distance,
            'stopNum': self.num_stops
        }
        point = Point(stop.location)
        feature = Feature(geometry=point, properties=stop_properties)
        self.geojson.append(feature)
        
        self.num_stops += 1
        self.distance += stop.distance
        self.total_duration += (stop.stop_duration + stop.driving_duration)
        self.idle_duration += stop.idle_duration
        self.driving_duration += stop.driving_duration
        self.stopped_duration += stop.stop_duration


In [5]:
class Department:
    def __init__ (
        self,
        name,
        vehicles
    ):
        self.name = name
        self.num_vehicles = len(vehicles);
        self.vehicles = vehicles


In [6]:
class Stop:
    def __init__(
        self,
        location_lat,
        location_long,
        stop_duration,
        idle_duration,
        driving_duration,
        start_time,
        end_time,
        distance
        ):
        self.location = (location_long, location_lat)
        self.stop_duration = stop_duration
        self.idle_duration = idle_duration
        self.driving_duration = driving_duration
        self.start_time = start_time
        self.end_time = end_time
        self.distance = distance

In [7]:
fleet_df = pd.read_csv('../src/data/vehicle_details/FleetDetails.csv')

In [8]:
fleet_df = fleet_df.fillna('')

In [9]:
fleet_df['VehicleName'] = [str(name).split(' ')[0] for name in fleet_df['VehicleName']]
fleet_df['VehicleName'] = [re.sub('[\W_]+', '', str(name)) for name in fleet_df['VehicleName']]

In [10]:
fleet = []
for idx, row in fleet_df.iterrows():
    vehicle = Vehicle(row['VehicleName'], row['Department'], row['Make'], row['Model'], row['Year'], row['Class'])
    fleet.append(vehicle)

In [11]:
depts = []
groups = fleet_df.groupby('Department')
for name, rows in groups:
    dept_vehicles = []
    for idx, row in rows.iterrows():
        dept_vehicles.append(row['VehicleName'])
    depts.append(Department(name, dept_vehicles))

In [12]:
trips_df = pd.read_csv('sept-oct_trip_latlng.csv')
# trips_df = trips_df.rename(columns={'Unnamed: 0': 'idx'})
# trips_df['trip_num'] = 0

In [13]:
# grouped = trips_df.groupby('Device')
# trip_num = 0
# for name, group in grouped:
#     last_row = group.iloc[0]
#     for i, row in group.iterrows():
#         if row.idx - last_row.idx > 1:
#             trip_num += 1
#         trips_df['trip_num'].iloc[row.idx] = trip_num
#         last_row = row
#     trip_num += 1

In [14]:
trip_objs = []
trips = []
grouped = trips_df.groupby('TripNum')
for name, group in grouped:
    trip = Trip(group.iloc[0]['Device'], group.iloc[0]['StopStart'], group.iloc[-1]['StopEnd'], group.iloc[0]['Date'])
    for i, row in group.iterrows():
        # if math.isnan(row.lat) or math.isnan(row.long):
        #     r_row = trips_df.sample(n=1).iloc[0]
        #     while not math.isnan(r_row.lat) and not math.isnan(r_row.long):
        #         r_row = trips_df.sample(n=1).iloc[0]
        #     lat = r_row.lat
        #     lng = r_row.long
        # else:
        #     lat = row.lat
        #     lng = row.long
        if not math.isnan(row.lat) and not math.isnan(row.long):
            stop = Stop(row['lat'], row['long'], row['StopDuration'], row['IdleDuration'], row['DrivingDuration'], row['StopStart'], row['StopEnd'], row['Distance'])
            trip.add_stop(stop)
    trips.append(trip)
    trip_objs.append(trip.__dict__)



In [15]:
for vehicle in fleet:
    for trip in trips:
        if vehicle.id == trip.vehicle:
            trip_obj = {
                'idle_duration': trip.idle_duration,
                'total_duration': trip.total_duration,
                'driving_duration': trip.driving_duration,
                'stopped_duration': trip.stopped_duration,
                'distance': trip.distance,
                'start_time': trip.start_time,
                'end_time': trip.end_time,
                'num_stops': trip.num_stops,
            }
            vehicle.trips.append(trip_obj)

In [16]:
for vehicle in fleet:
    vehicle.total_trips = 0
    vehicle.total_driving_duration = 0
    vehicle.total_distance = 0
    if len(vehicle.trips) == 0:
        continue
    for trip in vehicle.trips:
        vehicle.total_trips += 1
        vehicle.total_driving_duration += trip['driving_duration']
        vehicle.total_distance += trip['distance']

In [17]:
max_trips = max(fleet, key=attrgetter('total_trips'))
max_distance = max(fleet, key=attrgetter('total_distance'))
max_duration = max(fleet, key=attrgetter('total_driving_duration'))

for vehicle in fleet:
    vehicle.days_utilization = round(vehicle.total_trips / max_trips.total_trips, 3)
    vehicle.distance_utilization = round(vehicle.total_distance / max_distance.total_distance, 3)
    vehicle.duration_utilization = round(vehicle.total_driving_duration / max_duration.total_driving_duration, 3)
    vehicle.utilization_rate = round((vehicle.days_utilization + vehicle.distance_utilization + vehicle.duration_utilization) / 3, 3)


In [18]:
for dept in depts:
    dept_vehicles = []
    for vehicle_id in dept.vehicles:
        vehicle = [vehicle for vehicle in fleet if vehicle.id == vehicle_id]
        dept_vehicles.extend(vehicle)
    dept.avg_days_utilization = 0
    dept.avg_distance_utilization = 0
    dept.avg_duration_utilization = 0
    dept.avg_utilization_rate = 0
    for vehicle in dept_vehicles:
        dept.avg_days_utilization += vehicle.days_utilization
        dept.avg_distance_utilization += vehicle.distance_utilization
        dept.avg_duration_utilization += vehicle.duration_utilization
        dept.avg_utilization_rate += vehicle.utilization_rate
    dept.avg_days_utilization = round(dept.avg_days_utilization / len(dept_vehicles), 3)
    dept.avg_distance_utilization = round(dept.avg_distance_utilization / len(dept_vehicles), 3)
    dept.avg_duration_utilization = round(dept.avg_duration_utilization / len(dept_vehicles), 3)
    dept.avg_utilization_rate = round(dept.avg_utilization_rate / len(dept_vehicles), 3)



In [85]:
fleet_obj_arr = []
for vehicle in fleet:
    fleet_obj_arr.append(vehicle.__dict__)
    

In [86]:
dept_obj_arr = []
for dept in depts:
    dept_obj_arr.append(dept.__dict__)

In [19]:
trips_df

,Unnamed: 0,Device,DeviceGroup,Date,StopStart,DrivingDuration,StopEnd,Distance,StopDuration,Location,StopZoneType,IdleDuration,TripNum,NonTripStop,lat,long
0,0,FO15921,"Vehicle, Hospital ITS","September 1, 2022","Sep 01, 2022 10:33:43 AM",1,"Sep 01, 2022 10:35:12 AM",0.0,8.0,"75 W 900 S, Salt Lake City, UT 84101, USA",NaN,0,0,False,40.749723,-111.893200
1,1,FO15921,"Vehicle, Hospital ITS","September 1, 2022","Sep 01, 2022 10:43:34 AM",4,"Sep 01, 2022 10:47:35 AM",1.0,2.0,"175 E 800 S, Salt Lake City, UT 84111, USA",NaN,0,0,False,40.752524,-111.885770
2,2,FO15921,"Vehicle, Hospital ITS","September 1, 2022","Sep 01, 2022 10:49:47 AM",5,"Sep 01, 2022 10:55:27 AM",1.0,5.0,"102 S 200 E, Salt Lake City, UT 84111, USA",NaN,0,0,False,40.766793,-111.885979
3,3,FO15921,"Vehicle, Hospital ITS","September 1, 2022","Sep 01, 2022 11:01:15 AM",3,"Sep 01, 2022 11:04:17 AM",1.0,13.0,"530 E 100 S, Salt Lake City, UT 84102, USA",NaN,6,0,False,40.766809,-111.875623
4,4,FO15921,"Vehicle, Hospital ITS","September 1, 2022","Sep 01, 2022 11:17:48 AM",4,"Sep 01, 2022 11:21:55 AM",0.0,33.0,"555 100 S, Salt Lake City, UT 84102, USA",NaN,1,0,False,40.767653,-111.874850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34599,38681,UOFU6483,"Vehicle, Home Infusion","October 14, 2022","Oct 14, 2022 10:04:22 AM",20,"Oct 14, 2022 10:24:43 AM",14.0,7.0,"Campus: 635 Arapeen Dr, Salt Lake City, UT 841...",Campus,0,4082,False,40.615777,-111.889714
34600,38683,UOFU6483,"Vehicle, Home Infusion","October 20, 2022","Oct 20, 2022 12:00:40 PM",20,"Oct 20, 2022 12:21:12 PM",9.0,7.0,"1260 E Stringham Avenue South Suite 100, Salt ...",NaN,0,4083,False,40.273403,-111.697365
34601,38684,UOFU6483,"Vehicle, Home Infusion","October 20, 2022","Oct 20, 2022 12:28:44 PM",15,"Oct 20, 2022 12:43:45 PM",12.0,3.0,"897 E 6600 S, Murray, UT 84107, USA",NaN,0,4083,False,40.541889,-112.013329
34602,38686,UOFU6483,"Vehicle, Home Infusion","October 26, 2022","Oct 26, 2022 1:44:35 PM",27,"Oct 26, 2022 2:12:28 PM",13.0,8.0,"5223 Duckhorn Dr, South Jordan, UT 84095, USA",NaN,0,4084,False,40.540574,-112.015523


In [87]:
stops_obj_arr = []
for i, row in trips_df.iterrows():
    if not math.isnan(row.lat) and not math.isnan(row.long):
        stop = Stop(row['lat'], row['long'], row['StopDuration'], row['IdleDuration'], row['DrivingDuration'], row['StopStart'], row['StopEnd'], row['Distance'])
        stop.lat = row['lat']
        stop.lng = row['long']
        stop.device = row['Device']
        stop.group = row['DeviceGroup']
        stops_obj_arr.append(stop.__dict__)

In [88]:
stops_obj = {'stops': stops_obj_arr}

with open("heatmap_data.json", "w") as outfile:
    json.dump(stops_obj, outfile)

In [89]:
fleet_details_obj = {'vehicles': fleet_obj_arr}
trip_details_obj = {'trips': trip_objs}
dept_details_obj = {'departments': dept_obj_arr}

In [90]:
with open("fleet_details.json", "w") as outfile:
    json.dump(fleet_details_obj, outfile)

with open("trips_sept-oct.json", "w") as outfile:
    json.dump(trip_details_obj, outfile)

    
with open("depts_9-27_9-29.json", "w") as outfile:
    json.dump(dept_details_obj, outfile)